## Routing Research  

**Common Algorithms**  

1.  Dijksta's Algorithm
1.  A* Algorithm
1.  Bellman-Ford Algorithm
1.  Floyd-Warshall Algorithm
1.  Johnson's Algorithm (incomplete example)
1.  Bidirectional Search 
1.  Ant Colony Optimization (ACO)
1.  Genetic Algorithms  

In [1]:
import heapq
from collections import defaultdict

def dijkstra(graph, start_node):
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0
    
    queue = [(0, start_node)]
    
    while queue:
        current_distance, current_node = heapq.heappop(queue)
        
        # Skip if already visited
        if current_distance > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # Update the distance if a shorter path is found
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(queue, (distance, neighbor))
    
    return distances


# Example usage
graph = defaultdict(dict)
graph['A']['B'] = 5
graph['A']['C'] = 2
graph['B']['C'] = 1
graph['B']['D'] = 3
graph['C']['B'] = 1
graph['C']['D'] = 6
graph['C']['E'] = 4
graph['D']['E'] = 2
graph['E']['D'] = 2

start_node = 'A'
distances = dijkstra(graph, start_node)

# Print the shortest distances from the start node to all other nodes
for node, distance in distances.items():
    print(f"Shortest distance from {start_node} to {node}: {distance}")


Shortest distance from A to A: 0
Shortest distance from A to B: 3
Shortest distance from A to C: 2
Shortest distance from A to D: 6
Shortest distance from A to E: 6


In [7]:
# This is partially correct - but it does not check for graph consistency.
# An impossible or incomplete graph structure could be provided and it would not 
# detect the problem and still produce an answer.

# A* Algorithm

import heapq
from math import sqrt


class Node:
    def __init__(self, state, parent=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.g = g  # Cost from start node to current node
        self.h = h  # Heuristic estimate from current node to goal node

    def f(self):
        return self.g + self.h

    def __lt__(self, other):
        return self.f() < other.f()


def euclidean_distance(node1, node2, graph):
    x1, y1 = graph[node1]
    x2, y2 = graph[node2]
    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def a_star(start, goal, graph, heuristic):
    open_set = []
    closed_set = set()

    start_node = Node(start, None, 0, heuristic(start, goal, graph))
    heapq.heappush(open_set, start_node)

    while open_set:
        current_node = heapq.heappop(open_set)
        current_state = current_node.state

        if current_state == goal:
            path = []
            while current_node:
                path.append(current_node.state)
                current_node = current_node.parent
            return path[::-1]

        closed_set.add(current_state)

        for neighbor in graph[current_state]:
            if neighbor in closed_set:
                continue

            g = current_node.g + graph[current_state][neighbor]
            h = heuristic(neighbor, goal, graph)
            new_node = Node(neighbor, current_node, g, h)

            # Check if there is a better path to the neighbor
            is_better = True
            for open_node in open_set:
                if open_node.state == neighbor and open_node.f() <= new_node.f():
                    is_better = False
                    break

            if is_better:
                heapq.heappush(open_set, new_node)

    return None


# Example usage
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'C': 1, 'D': 3},
    'C': {'B': 1, 'D': 6, 'E': 4},
    'D': {'E': 2},
    'E': {'C':4, 'D': 2}
}

# Node locations (x, y)
node_locations = {
    'A': (0, 0),
    'B': (1, 1),
    'C': (2, 2),
    'D': (3, 1),
    'E': (4, 0)
}

start_node = 'A'
goal_node = 'E'

path = a_star(start_node, goal_node, graph, lambda n1, n2, g: euclidean_distance(n1, n2, node_locations))

if path:
    print("Shortest path found:")
    print(" -> ".join(path))
else:
    print("No path found.")



Shortest path found:
A -> C -> E


In [12]:
# Bellman-Ford Algorithm
class Edge:
    def __init__(self, source, destination, weight):
        self.source = source
        self.destination = destination
        self.weight = weight


def bellman_ford(graph, start_node):
    # Step 1: Initialization
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0

    # Step 2: Relaxation
    for _ in range(len(graph.edges)-1):
        for edge in graph.edges:
            u = edge.source
            v = edge.destination
            w = edge.weight
            if distances[u] != float('inf') and distances[u] + w < distances[v]:
                distances[v] = distances[u] + w

    # Step 3: Check for negative cycles
    for edge in graph.edges:
        u = edge.source
        v = edge.destination
        w = edge.weight
        if distances[u] != float('inf') and distances[u] + w < distances[v]:
            raise ValueError("Graph contains a negative cycle")

    return distances


# Example usage
class Graph:
    def __init__(self):
        self.edges = []

    def add_edge(self, source, destination, weight):
        edge = Edge(source, destination, weight)
        self.edges.append(edge)
        

    def __iter__(self):
        return iter(set(edge.source for edge in self.edges) |
                    set(edge.destination for edge in self.edges))


graph = Graph()
graph.add_edge('A', 'B', 5)
graph.add_edge('A', 'C', 2)
graph.add_edge('B', 'C', 1)
graph.add_edge('B', 'D', 3)
graph.add_edge('C', 'B', 1)
graph.add_edge('C', 'D', 6)
graph.add_edge('C', 'E', 4)
graph.add_edge('D', 'E', 2)
graph.add_edge('E', 'D', 2)

start_node = 'A'
distances = bellman_ford(graph, start_node)

# Print the shortest distances from the start node to all other nodes
for node, distance in distances.items():
    print(f"Shortest distance from {start_node} to {node}: {distance}")


Shortest distance from A to E: 6
Shortest distance from A to C: 2
Shortest distance from A to A: 0
Shortest distance from A to B: 3
Shortest distance from A to D: 6


In [13]:
# Floyd-Warshall Algorithm
INF = float('inf')

def floyd_warshall(graph):
    num_vertices = len(graph)
    distances = [[0 if i == j else graph[i][j] if graph[i][j] != 0 else INF for j in range(num_vertices)] for i in range(num_vertices)]

    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])

    return distances


# Example usage
graph = [
    [0, 5, INF, INF, INF],
    [INF, 0, 1, 3, INF],
    [INF, 1, 0, INF, 6],
    [INF, 3, INF, 0, 2],
    [INF, INF, 6, 2, 0]
]

distances = floyd_warshall(graph)

# Print the shortest distances between all pairs of vertices
for i in range(len(graph)):
    for j in range(len(graph)):
        if distances[i][j] == INF:
            print(f"No path from vertex {i} to vertex {j}")
        else:
            print(f"Shortest distance from vertex {i} to vertex {j}: {distances[i][j]}")


Shortest distance from vertex 0 to vertex 0: 0
Shortest distance from vertex 0 to vertex 1: 5
Shortest distance from vertex 0 to vertex 2: 6
Shortest distance from vertex 0 to vertex 3: 8
Shortest distance from vertex 0 to vertex 4: 10
No path from vertex 1 to vertex 0
Shortest distance from vertex 1 to vertex 1: 0
Shortest distance from vertex 1 to vertex 2: 1
Shortest distance from vertex 1 to vertex 3: 3
Shortest distance from vertex 1 to vertex 4: 5
No path from vertex 2 to vertex 0
Shortest distance from vertex 2 to vertex 1: 1
Shortest distance from vertex 2 to vertex 2: 0
Shortest distance from vertex 2 to vertex 3: 4
Shortest distance from vertex 2 to vertex 4: 6
No path from vertex 3 to vertex 0
Shortest distance from vertex 3 to vertex 1: 3
Shortest distance from vertex 3 to vertex 2: 4
Shortest distance from vertex 3 to vertex 3: 0
Shortest distance from vertex 3 to vertex 4: 2
No path from vertex 4 to vertex 0
Shortest distance from vertex 4 to vertex 1: 5
Shortest distance

In [18]:
# Johnson's Algorithm  - needs work
import heapq
from collections import defaultdict

INF = float('inf')


def johnson(graph):
    num_vertices = len(graph)

    # Step 1: Add an extra vertex with zero-weight edges to all other vertices
    augmented_graph = add_extra_vertex(graph)

    # Step 2: Run Bellman-Ford algorithm to find potentials (h values)
    h = bellman_ford(augmented_graph, num_vertices)

    if h is None:
        return None

    # Step 3: Re-weight the edges based on the potentials
    reweighted_graph = reweight_edges(graph, h)

    # Step 4: Run Dijkstra's algorithm for each vertex to find shortest paths
    distances = []
    for u in range(num_vertices):
        dist_u = dijkstra(reweighted_graph, u)
        distances.append(dist_u)

    # Step 5: Revert the re-weighting using the potentials
    distances = revert_reweighting(distances, h)

    return distances


def add_extra_vertex(graph):
    num_vertices = len(graph)
    augmented_graph = graph + [[0] * num_vertices]
    augmented_graph.append([0] * (num_vertices + 1))
    return augmented_graph


def bellman_ford(graph, start_node):
    num_vertices = len(graph)
    distances = [INF] * num_vertices
    distances[start_node] = 0

    for _ in range(num_vertices - 1):
        for u in range(num_vertices):
            for v in range(num_vertices):
                if graph[u][v] != 0 and distances[u] != INF and distances[u] + graph[u][v] < distances[v]:
                    distances[v] = distances[u] + graph[u][v]

    for u in range(num_vertices):
        for v in range(num_vertices):
            if graph[u][v] != 0 and distances[u] != INF and distances[u] + graph[u][v] < distances[v]:
                return None

    return distances


def reweight_edges(graph, h):
    num_vertices = len(graph)
    reweighted_graph = [[0] * num_vertices for _ in range(num_vertices)]

    for u in range(num_vertices):
        for v in range(num_vertices):
            if graph[u][v] != 0:
                reweighted_graph[u][v] = graph[u][v] + h[u] - h[v]

    return reweighted_graph


def dijkstra(graph, start_node):
    num_vertices = len(graph)
    distances = [INF] * num_vertices
    distances[start_node] = 0

    heap = [(0, start_node)]

    while heap:
        dist, u = heapq.heappop(heap)

        if dist > distances[u]:
            continue

        for v in range(num_vertices):
            if graph[u][v] != 0:
                new_dist = dist + graph[u][v]
                if new_dist < distances[v]:
                    distances[v] = new_dist
                    heapq.heappush(heap, (new_dist, v))

    return distances


def revert_reweighting(distances, h):
    num_vertices = len(distances)

    for u in range(num_vertices):
        for v in range(num_vertices):
            if distances[u][v] != INF:
                distances[u][v] = distances[u][v] - h[u] + h[v]

    return distances


# Example usage
graph = [
    [0, -2, 3, 0],
    [4, 0, 0, 1],
    [0, -1, 0, 2],
    [0, 0, 0, 0]
]

distances = johnson(graph)

if distances is None:
    print("Graph contains a negative cycle")
else:
    num_vertices = len(graph)
    for u in range(num_vertices):
        for v in range(num_vertices):
            print(f"Shortest distance from vertex {u} to vertex {v}: {distances[u][v]}")

IndexError: list index out of range

In [19]:
# Bidirection search
from collections import deque


def bidirectional_search(graph, start_node, end_node):
    if start_node == end_node:
        return [start_node]

    forward_queue = deque([(start_node, [start_node])])
    backward_queue = deque([(end_node, [end_node])])
    forward_visited = set([start_node])
    backward_visited = set([end_node])

    while forward_queue and backward_queue:
        forward_node, forward_path = forward_queue.popleft()
        backward_node, backward_path = backward_queue.popleft()

        # Explore neighbors in forward direction
        for neighbor in graph[forward_node]:
            if neighbor not in forward_visited:
                forward_visited.add(neighbor)
                new_path = forward_path + [neighbor]
                forward_queue.append((neighbor, new_path))

                # Check for intersection with backward search
                if neighbor in backward_visited:
                    intersection_node = neighbor
                    intersection_path = backward_path[::-1] + new_path[1:]
                    return intersection_path

        # Explore neighbors in backward direction
        for neighbor in graph[backward_node]:
            if neighbor not in backward_visited:
                backward_visited.add(neighbor)
                new_path = [neighbor] + backward_path
                backward_queue.append((neighbor, new_path))

                # Check for intersection with forward search
                if neighbor in forward_visited:
                    intersection_node = neighbor
                    intersection_path = forward_path + new_path[1:]
                    return intersection_path

    return None


# Example usage
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E', 'G'],
    'G': ['F']
}

start = 'A'
end = 'G'

path = bidirectional_search(graph, start, end)

if path is None:
    print("No path exists between the start and end nodes.")
else:
    print("Shortest path:", path)


Shortest path: ['A', 'B', 'F', 'G']


In [20]:
# Ant Colony Optimization

import random


class Ant:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.tabu_list = [False] * num_cities
        self.visited_cities = []
        self.total_distance = 0.0

    def choose_next_city(self, pheromone_matrix, distance_matrix, alpha, beta):
        current_city = self.visited_cities[-1]
        unvisited_cities = [i for i in range(self.num_cities) if not self.tabu_list[i]]

        if not unvisited_cities:
            return None

        probabilities = []
        for city in unvisited_cities:
            pheromone = pheromone_matrix[current_city][city]
            distance = distance_matrix[current_city][city]
            attractiveness = (pheromone ** alpha) * ((1.0 / distance) ** beta)
            probabilities.append(attractiveness)

        total_prob = sum(probabilities)
        probabilities = [p / total_prob for p in probabilities]
        next_city = random.choices(unvisited_cities, probabilities)[0]

        return next_city

    def visit_city(self, city, distance):
        self.tabu_list[city] = True
        self.visited_cities.append(city)
        self.total_distance += distance


class ACO:
    def __init__(self, num_ants, num_iterations, alpha, beta, rho, q0):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q0 = q0

    def optimize(self, distance_matrix):
        num_cities = len(distance_matrix)
        pheromone_matrix = [[1.0] * num_cities for _ in range(num_cities)]
        best_distance = float('inf')
        best_path = []

        for _ in range(self.num_iterations):
            ants = [Ant(num_cities) for _ in range(self.num_ants)]
            for ant in ants:
                ant.visit_city(random.randint(0, num_cities - 1), 0.0)

            for _ in range(num_cities - 1):
                for ant in ants:
                    current_city = ant.visited_cities[-1]
                    next_city = ant.choose_next_city(pheromone_matrix, distance_matrix, self.alpha, self.beta)
                    if next_city is None:
                        break
                    distance = distance_matrix[current_city][next_city]
                    ant.visit_city(next_city, distance)

            for ant in ants:
                ant.visit_city(ant.visited_cities[0], distance_matrix[ant.visited_cities[-1]][ant.visited_cities[0]])

                if ant.total_distance < best_distance:
                    best_distance = ant.total_distance
                    best_path = ant.visited_cities[:]

            for i in range(num_cities):
                for j in range(num_cities):
                    if i != j:
                        pheromone_matrix[i][j] *= (1.0 - self.rho)

            for ant in ants:
                delta_pheromone = 1.0 / ant.total_distance
                for i in range(num_cities - 1):
                    pheromone_matrix[ant.visited_cities[i]][ant.visited_cities[i + 1]] += delta_pheromone

            pheromone_matrix = [[max(pheromone, 1e-10) for pheromone in row] for row in pheromone_matrix]

        return best_path, best_distance


# Example usage
# Define the distance matrix (replace with your own data)
distance_matrix = [
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
]

num_ants = 10
num_iterations = 100
alpha = 1.0  # Pheromone exponent
beta = 2.0  # Distance exponent
rho = 0.5  # Pheromone evaporation rate
q0 = 0.5  # Exploration factor

aco = ACO(num_ants, num_iterations, alpha, beta, rho, q0)
best_path, best_distance = aco.optimize(distance_matrix)

print("Best Path:", best_path)
print("Best Distance:", best_distance)


Best Path: [3, 1, 0, 2, 3]
Best Distance: 21.0


In [21]:
# Genetic Algorithm for Routing
import random


class Route:
    def __init__(self, cities):
        self.cities = cities
        self.distance = self.calculate_distance()

    def calculate_distance(self):
        total_distance = 0
        for i in range(len(self.cities) - 1):
            current_city = self.cities[i]
            next_city = self.cities[i + 1]
            total_distance += distance_matrix[current_city][next_city]
        return total_distance


def create_initial_population(num_routes, num_cities):
    population = []
    for _ in range(num_routes):
        cities = random.sample(range(num_cities), num_cities)
        route = Route(cities)
        population.append(route)
    return population


def selection(population, num_parents):
    parents = sorted(population, key=lambda x: x.distance)[:num_parents]
    return parents


def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1.cities) - 1)
    child_cities = parent1.cities[:crossover_point] + [city for city in parent2.cities if city not in parent1.cities[:crossover_point]]
    child = Route(child_cities)
    return child


def mutation(route):
    mutation_point1, mutation_point2 = random.sample(range(len(route.cities)), 2)
    route.cities[mutation_point1], route.cities[mutation_point2] = route.cities[mutation_point2], route.cities[mutation_point1]
    route.distance = route.calculate_distance()


def genetic_algorithm(num_cities, num_routes, num_generations):
    population = create_initial_population(num_routes, num_cities)

    for _ in range(num_generations):
        parents = selection(population, num_parents=2)

        offspring = []
        while len(offspring) < num_routes - len(parents):
            parent1, parent2 = random.choices(parents, k=2)
            child = crossover(parent1, parent2)
            offspring.append(child)

        population = parents + offspring

        for route in population:
            if random.random() < mutation_rate:
                mutation(route)

    best_route = min(population, key=lambda x: x.distance)
    return best_route


# Example usage
# Define the distance matrix
distance_matrix = [
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
]

num_cities = len(distance_matrix)
num_routes = 50
num_generations = 100
mutation_rate = 0.01

best_route = genetic_algorithm(num_cities, num_routes, num_generations)

print("Best Route:", best_route.cities)
print("Best Distance:", best_route.distance)


Best Route: [2, 3, 1, 0]
Best Distance: 12


In [33]:
import math

graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'C': 1, 'D': 3},
    'C': {'B': 1, 'D': 6, 'E': 4},
    'D': {'E': 2},
    'E': {'C': 4, 'D': 2}
}

# Node locations (x, y)
node_locations = {
    'A': (0, 0),
    'B': (1, 1),
    'C': (2, 2),
    'D': (3, 1),
    'E': (4, 0)
}

# Calculate distance matrix
num_cities = len(graph)
distance_matrix = [[0.0] * num_cities for _ in range(num_cities)]
matrix_labels = [[''] * num_cities for _ in range(num_cities)]
city_to_node = {}

for i, city1 in enumerate(graph):
    city_to_node[i] = city1
    for j, city2 in enumerate(graph):
        x1, y1 = node_locations[city1]
        x2, y2 = node_locations[city2]
        distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        distance_matrix[i][j] = distance
        matrix_labels[i][j] = f'{city1}-{city2}'

# Print distance matrix
for row in distance_matrix:
    print(row)
    
for row in matrix_labels:
    print(row)

for k,v in city_to_node.items():
    print(k,v)

[0.0, 1.4142135623730951, 2.8284271247461903, 3.1622776601683795, 4.0]
[1.4142135623730951, 0.0, 1.4142135623730951, 2.0, 3.1622776601683795]
[2.8284271247461903, 1.4142135623730951, 0.0, 1.4142135623730951, 2.8284271247461903]
[3.1622776601683795, 2.0, 1.4142135623730951, 0.0, 1.4142135623730951]
[4.0, 3.1622776601683795, 2.8284271247461903, 1.4142135623730951, 0.0]
['A-A', 'A-B', 'A-C', 'A-D', 'A-E']
['B-A', 'B-B', 'B-C', 'B-D', 'B-E']
['C-A', 'C-B', 'C-C', 'C-D', 'C-E']
['D-A', 'D-B', 'D-C', 'D-D', 'D-E']
['E-A', 'E-B', 'E-C', 'E-D', 'E-E']
0 A
1 B
2 C
3 D
4 E


In [34]:
# Example usage
# Using above calculations
num_cities = len(distance_matrix)
num_routes = 50
num_generations = 100
mutation_rate = 0.01

best_route = genetic_algorithm(num_cities, num_routes, num_generations)

print("Best Route:", best_route.cities)
print(f"Best Route by Node Letter: {[city_to_node[i] for i in best_route.cities]}")
print("Best Distance:", best_route.distance)

Best Route: [4, 3, 2, 1, 0]
Best Route by Node Letter: ['E', 'D', 'C', 'B', 'A']
Best Distance: 5.656854249492381
